In [11]:
pip install splink

### Splink Setup

In [12]:
from splink.spark.jar_location import similarity_jar_location

from pyspark  import SparkContext, SparkConf
from pyspark.sql import SparkSession, types
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql.functions import substring
import splink.spark.comparison_library as cl
import splink.spark.comparison_level_library as cll
import splink.spark.comparison_template_library as ctl
import splink.spark.blocking_rule_library as brl
from splink.comparison import Comparison
from splink.spark.linker import SparkLinker
import os
import pprint

conf = SparkConf()


conf.set("spark.driver.memory", "64g")
conf.set("spark.kryoserializer.buffer.max","1024m")
conf.set("spark.default.parallelism", "40")
spark.conf.set("spark.sql.shuffle.partitions", "40")


# Adds custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
# The jar file needs to be downloaded from the above and uploaded to the synapse workspace
path = similarity_jar_location()

#conf.set('spark.driver.extraClassPath', path) #added by ash
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

os.makedirs("/tmp/Temp", exist_ok=True)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Data Matching Function Setup

In [13]:
spark.udf.registerJavaFunction('jaro_winkler', 'uk.gov.moj.dash.linkage.JaroWinklerSimilarity',DoubleType())                              
spark.udf.registerJavaFunction('jaccard_sim', 'uk.gov.moj.dash.linkage.JaccardSimilarity',DoubleType())                          
spark.udf.registerJavaFunction('cosine_distance', 'uk.gov.moj.dash.linkage.CosineDistance',DoubleType())
spark.udf.registerJavaFunction('sqlEscape', 'uk.gov.moj.dash.linkage.sqlEscape',StringType())                        
spark.udf.registerJavaFunction('levdamerau_distance', 'uk.gov.moj.dash.linkage.LevDamerauDistance',DoubleType())   
spark.udf.registerJavaFunction('jaro_sim', 'uk.gov.moj.dash.linkage.JaroSimilarity',DoubleType())   

### Drive Mounting

In [14]:
#if not(any(mount.mountPoint == "/lake" for mount in mssparkutils.fs.mounts())):
#    mssparkutils.fs.mount(
#        "abfss://synapse@sasynapsesplinkproduks01.dfs.core.windows.net",
#        "/lake",
#        {"linkedService" : "synw-synapsesplink-prod-uks-01-WorkspaceDefaultStorage"}
#    )

#    MountPath = "synfs:/" + mssparkutils.env.getJobId() + "/lake/splink/"
#    print(MountPath)
#else:
#    print(MountPath)

### Import Demo Dataset

In [15]:
#https://moj-analytical-services.github.io/splink/datasets.html

from splink.datasets import splink_datasets

#df = splink_datasets.fake_1000
#columns: ['unique_id', 'first_name', 'surname', 'dob', 'city', 'email', 'cluster']

pdf = splink_datasets.historical_50k
df = spark.createDataFrame(pdf)
#columns: ['unique_id','cluster','full_name','first_and_surname','first_name','surname','dob','birth_place','postcode_fake','gender','occupation']

#list(df.columns.values)
#pdf.head(5)
#df.limit(5).show()

df.count()


### Linkage settings

In [16]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules_to_generate_predictions": [
        #brl.block_on(["surname"]),
        brl.block_on(["first_name","surname","dob"]),
        brl.block_on(["first_name","surname","postcode_fake"]),
        brl.block_on(["first_name","dob","postcode_fake"]),
        brl.block_on(["surname","dob","postcode_fake"]),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
    #"additional_columns_to_retain": ["Street","Locality","Town","County"]
    #"em_convergence": 0.001
}


### Create the linkage model

In [17]:
linker = SparkLinker(df, settings)

linker.profile_columns(
    ["first_name", "surname", "postcode_fake", "substr(dob, 1,4)"], top_n=10, bottom_n=5
)

### Number of comparisons generated by deterministic rules

In [18]:
linker.cumulative_num_comparisons_from_blocking_rules_chart()


### Generate comparisons

In [19]:
df_predict = linker.deterministic_link()
df_predict.as_pandas_dataframe().head()


### Cluster to generate distinct output list

In [20]:
clusters = linker.cluster_pairwise_predictions_at_threshold(df_predict, threshold_match_probability=1)
#cdf = clusters.as_pandas_dataframe
#clusters.as_pandas_dataframe(limit=5)
spark.createDataFrame(clusters.as_pandas_dataframe()).createOrReplaceTempView("clusters")

In [21]:
%%sql

select *
from clusters
order by cluster_id
limit 20

In [22]:
%%sql
select count(1)
from
(select distinct cluster_id
from clusters)
--limit 10